In [4]:
import json
import pandas as pd
import gzip
import numpy as np
from sklearn.externals import joblib
from github import Github
import gensim
from langdetect import detect
from collections import Counter

In [3]:
readMe = ["THis ia yes another string","THis is another string","THis is a string"]
readMe = [0]
len(readMe)

1

In [5]:
# I stole this from Amy's code to read in the data:
events = []
# File from: wget http://data.gharchive.org/2018-07-06-1.json.gz
# i.e. archive for 24 hours on 7/06/18
for hour in range(0, 23):
    with gzip.open('/Users/shaunogrady/MetaGithub/July6Data/2018-07-06-%d.json.gz' %hour) as f:
        for line in f:
            events.append(json.loads(line.strip()))

In [6]:
Counter((event['type'] for event in events));

In [7]:
#issues = [event for event in events if event['type'] == 'IssuesEvent']
#comments = [event for event in events if event['type'] == 'IssueCommentEvent']
forkEvents = [event for event in events if event['type'] == 'ForkEvent']
#CreateEvents = [event for event in events if event['type'] == 'CreateEvent']

In [8]:
#Hold a list of all the usernames and repo names
#This is needed in order to pull README.md files
names=[]
## pull all of the createEvents in order to get UserID and Repo name for 
# calls to get Readme Content
for event in range(0,len(forkEvents)):
    thisEvent= forkEvents[event]
    thisRepo = thisEvent['repo']
    names.append(thisRepo['name'])

In [6]:
len(names)

33165

In [9]:
## gets rid of doubles:
names = np.array(names)
unique, counts = np.unique(names, return_counts=True)
readMesAndPopularity = pd.DataFrame(
    {'usernameRepoName': unique,
     'num_forks': counts
    })
# sort by number of forks in the date/time range:
readMesAndPopularity = readMesAndPopularity.sort_values(by='num_forks',ascending=False)
# 10000 most popular repos based on number of forks in the date/time range:
readMesAndPopularity = readMesAndPopularity[:10000]

In [16]:
g = Github("6efe0a63859ff9f96f2e1b829db3525b33d2f533")
reponames = readMesAndPopularity['usernameRepoName']
# total number of forks that this repo has:
forks = []
# Total number of users watching this repo:
watchers = []
# Total number of users that starred this repo:
stargazers = []
# total number of useres subscribed to this repo:
subscribers = []
# Read Me strings
ReadMeFiles = []
# Language of readMes:
langs = []
for repoName in range(0,len(reponames)-1):
    thisRepo=g.get_repo(reponames.iloc[repoName])
    try:
        forks.append(thisRepo.forks_count)
    except:
        forks.append(0)
    try:
        watchers.append(thisRepo.watchers_count)
    except:
        watchers.append(0)
    try:
        stargazers.append(thisRepo.stargazers_count)
    except:
        stargazers.append(0)
    try:
        subscribers.append(thisRepo.subscribers_count)
    except:
        subscribers.append(0)
    try:
        readMe=thisRepo.get_contents('README.md')
        readMe = readMe.decoded_content
        readMe = readMe.decode("utf-8")
        ReadMeFiles.append(readMe)
        lang = detect(readMe)
        langs.append(lang)
    except:
        readMe ="No README.md found."
        ReadMeFiles.append(readMe)
        lang = detect(readMe)
        langs.append(lang)

In [17]:
len(ReadMeFiles)

802

In [80]:
## create a pandas dataframe holding all the variables for tracking the 
# 'popularity' measures of the repos plus the readMe files as strings:
readMesAndPopularity = pd.DataFrame(
    {'username/repoName': unique,
     'num_forks_dateRange': counts,
     'total_forks': forks,
     'stargazers': stargazers,
     'subscribers': subscribers,
     'readMeLanguage': langs,
     'readMes': ReadMeFiles
    })

ValueError: arrays must all be same length

In [71]:
reponames.iloc[9999]
ReadMeFiles[10005]

"A Practical Guide for Debugging TensorFlow Codes\n================================================\n\nMy talk in the first meetup of [TensorFlow Korea Group](https://www.facebook.com/groups/TensorFlowKR/)!\n\n18 June 2016, at Google Korea, Seoul\n\n- Slides: https://wookayin.github.io/tensorflow-talk-debugging\n- Codes: https://github.com/wookayin/tensorflow-talk-debugging/tree/master/codes\n- Video (in Korean): https://www.youtube.com/watch?v=01nDEZe0BfU\n\n## Updates\n\n- June 18th, 2016 (v1): Initial version of the talk.\n- December 9th, 2016 (v2): Added a brief introduction to the [official debugger][tfdbg], for readers' reference.\n- Feb 17th, 2016 (v3): Updated examples to be compatible with TF 1.0, and added more contents on [tfdbg][tfdbg].\n\n[tfdbg]: https://www.tensorflow.org/programmers_guide/debugger\n"

In [20]:
len(subscribers)

10000

In [62]:
joblib.dump(readMesAndPopularity, "readMeFiles.pkl")

['readMeFiles.pkl']

In [3]:
readMeFiles = joblib.load("readMeFiles.pkl")

In [9]:
files=readMeFiles.ReadMes
file = files[1]
file = file.decode("utf-8")
from langdetect import detect
lang = detect(file)
lang

'en'